import all libraries used

In [1]:
import pandas as pd
import aiohttp
import asyncio
import os
import re

async def download_image(session, url, filename, index, total_rows, stats):
    full_path = os.path.join(output_folder, filename)
    try:
        async with session.get(url) as response:
            response.raise_for_status()
            with open(full_path, 'wb') as f:
                while chunk := await response.content.read(32768):
                    f.write(chunk)
            stats['downloaded'] += 1
    except aiohttp.ClientError as err:
        # print(f"Something went wrong: {err}")
        stats['errors'] += 1

output_folder = 'Gaiat products'
os.makedirs(output_folder, exist_ok=True)
pattern = re.compile(r'[^\w\s-]')
df = pd.read_csv('D:\PENNY\Gaiat products.csv')
total_rows = len(df)

async def main():
    stats = {'downloaded': 0, 'skipped': 0, 'errors': 0}
    async with aiohttp.ClientSession() as session:
        tasks = []
        for index, row in df.iterrows():
            url = row['Image Links']
            filename = row['English Name (required)']
            # Skip if URL or filename is missing or not a string
            if not isinstance(url, str) or pd.isna(url) or not isinstance(filename, str) or pd.isna(filename):
                stats['skipped'] += 1
                continue
            filename = pattern.sub('', filename).replace(' ', '_') + '.jpg'
            task = download_image(session, url, filename, index, total_rows, stats)
            tasks.append(task)
        await asyncio.gather(*tasks)
        
    print(f"Summary: Downloaded {stats['downloaded']} images, skipped {stats['skipped']} images, encountered {stats['errors']} errors.")


In [2]:
import os
import concurrent.futures
from PIL import Image, ImageFilter

def process_image(filename):
    try:
        img = Image.open(filename)
        img_filtered = img.filter(ImageFilter.EDGE_ENHANCE)
        processed_filename = 'processed_' + os.path.basename(filename)
        img_filtered.save(os.path.join(folder_path, processed_filename))
    except (OSError, IOError):
        print(f"{filename} is broken and will be skipped.")

if __name__ == "__main__":
    # Folder path
    folder_path = r'D:\PENNY\Gaiat products' 

    # List all image files in the folder
    image_files = [os.path.join(folder_path, filename) for filename in os.listdir(folder_path) if filename.endswith('.jpg')]

    with concurrent.futures.ThreadPoolExecutor() as executor:
        executor.map(process_image, image_files)


D:\PENNY\Gaiat products\AirbodyDummy_Junior.jpg is broken and will be skipped.
D:\PENNY\Gaiat products\AirbodyDummy_Senior.jpg is broken and will be skipped.
D:\PENNY\Gaiat products\AI_Builder_Capacity_add-on_for_GCC_Microsoft.jpg is broken and will be skipped.
D:\PENNY\Gaiat products\AI_Builder_Capacity_add-on_for_Faculty_Microsoft.jpg is broken and will be skipped.
D:\PENNY\Gaiat products\Air_Freshener_Bottle_-_XD353N.jpg is broken and will be skipped.
D:\PENNY\Gaiat products\App_-_Almas_Containers_20L_-_12_Pcs_Multiple_colors.jpg is broken and will be skipped.
D:\PENNY\Gaiat products\App_-_Almas_Containers_5L_-_30_Pcs_Multiple_colors.jpg is broken and will be skipped.
D:\PENNY\Gaiat products\App_-_Almas_Containers_45L_-_12_Pcs_Multiple_colors.jpg is broken and will be skipped.
D:\PENNY\Gaiat products\App_-_Almas_Containers_10L_-_24_Pcs_Multiple_colors.jpg is broken and will be skipped.
D:\PENNY\Gaiat products\App_-_Altaj_Baby_Chair_-_12_Pcs_Multiple_colors.jpg is broken and will be 

In [2]:
pip install tqdm 


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



     ---------------------------------------- 77.1/77.1 kB 1.4 MB/s eta 0:00:00


In [3]:
import os
import concurrent.futures
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm  # Import tqdm for the progress bar

def process_batch(image_files, target_size=(224, 224)):
    batch_images = []
    for filename in image_files:
        try:
            img = cv2.imread(filename)
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img_filtered = cv2.filter2D(img_rgb, -1, np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]]))
            img_resized = cv2.resize(img_filtered, target_size)
            batch_images.append(img_resized)
        except (OSError, IOError, Exception) as e:
            print(f"Error processing {filename}: {e}")
    return batch_images

if __name__ == "__main__":
    # Folder path
    folder_path = r'D:\PENNY\Gaiat products' 

    # List all image files in the folder
    image_files = [os.path.join(folder_path, filename) for filename in os.listdir(folder_path) if filename.endswith('.jpg')]

    batch_size = 32
    num_images = len(image_files)

    with concurrent.futures.ProcessPoolExecutor() as executor:
        image_arrays = []
        with tqdm(total=num_images, desc="Processing Images", unit="images") as pbar:
            for i in range(0, num_images, batch_size):
                batch_images = process_batch(image_files[i:i + batch_size])
                image_arrays.extend(batch_images)
                pbar.update(len(batch_images))  # Update the progress bar

    # Remove any None elements (corresponding to broken images)
    image_arrays = [img_array for img_array in image_arrays if img_array is not None]

    # Prepare the dataset for classification
    X = np.stack(image_arrays)  # Convert list of arrays to numpy array
    y = np.ones(X.shape[0])  # Assuming all images belong to the same class (label 1)

    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Now you have X_train, y_train, X_test, and y_test ready for training your machine learning model.
    # You can use these arrays to train and evaluate your classification model.


Processing Images:   1%|          | 64/10286 [00:01<04:01, 42.38images/s]

Error processing D:\PENNY\Gaiat products\Ali_Café_3_In_1_Classic_Instant_Coffee_-_22_Sticks.jpg: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'

Error processing D:\PENNY\Gaiat products\Ali_Café_3_In_1_Instant_French_Coffee_-_20Packs_-24_Sticks.jpg: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'



Processing Images:   3%|▎         | 286/10286 [00:05<03:18, 50.38images/s]

Error processing D:\PENNY\Gaiat products\Bn130_Brown_Paper_Napkin_-_30Cm _40_Packs.jpg: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'



Processing Images:   5%|▍         | 509/10286 [00:12<06:29, 25.12images/s]

...

Processing Images:  90%|█████████ | 9289/10286 [04:02<00:27, 36.57images/s]

Error processing D:\PENNY\Gaiat products\Sharpened_Crème_Caramel_-_72_Pcs_-100g.jpg: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'



Processing Images:  97%|█████████▋| 9927/10286 [04:13<00:05, 63.50images/s]

Error processing D:\PENNY\Gaiat products\Sharpened_Nescafé_Classic_3_In1_Instant_Coffee_-_12Packs_-28_Sticks.jpg: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'

Error processing D:\PENNY\Gaiat products\Sharpened_Nescafé_Classic_Instant_Coffee_-_12_Bottles.jpg: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'



Processing Images:  99%|█████████▉| 10197/10286 [04:19<00:02, 39.23images/s]


In [18]:
import os
import concurrent.futures
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm  # Import tqdm for the progress bar
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

def process_image(filename, target_size=(224, 224)):
    try:
        img = cv2.imread(filename)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_filtered = cv2.filter2D(img_rgb, -1, np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]]))
        img_resized = cv2.resize(img_filtered, target_size)
        img_array = img_resized / 255.0  # Normalize pixel values to [0, 1]
        return img_array
    except (OSError, IOError, Exception) as e:
        print(f"Error processing {filename}: {e}")
        return None

def load_data(folder_path, target_size=(224, 224)):
    image_files = [os.path.join(folder_path, filename) for filename in os.listdir(folder_path) if filename.endswith('.jpg')]
    X = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        with tqdm(total=len(image_files), desc="Processing Images", unit="images") as pbar:
            for img_array in executor.map(process_image, image_files):
                if img_array is not None:
                    X.append(img_array)
                pbar.update(1)  # Update the progress bar

    return np.array(X)

if __name__ == "__main__":
    # Folder path
    folder_path = r'D:\PENNY\Gaiat products'

    # Load the image data
    X = load_data(folder_path)

    # Load the Catalog Names and Category Names (readonly)
    # Replace the following lists with your actual catalog and category names
    catalog_names = ["Stationery", "Electronics", "Clothing", ...]
    category_names = ["Files", "Laptops", "T-Shirts", ...]

    # Prepare the labels for catalog and category classification
    y_catalog = np.random.randint(len(catalog_names), size=X.shape[0])
    y_category = np.random.randint(len(category_names), size=X.shape[0])

    # Split the dataset into training and testing sets
    X_train, X_test, y_train_catalog, y_test_catalog, y_train_category, y_test_category = train_test_split(
        X, y_catalog, y_category, test_size=0.2, random_state=42)

    # Load pre-trained VGG16 model
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    # Add new classification layers for catalog and category
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    catalog_output = Dense(len(catalog_names), activation='softmax', name='catalog_output')(x)  # Output layer for catalog classification
    category_output = Dense(len(category_names), activation='softmax', name='category_output')(x)  # Output layer for category classification

    model = Model(inputs=base_model.input, outputs=[catalog_output, category_output])

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss={'catalog_output': 'sparse_categorical_crossentropy', 'category_output': 'sparse_categorical_crossentropy'},
                  metrics={'catalog_output': 'accuracy', 'category_output': 'accuracy'})

    # Train the model
    model.fit(X_train, {'catalog_output': y_train_catalog, 'category_output': y_train_category},
              validation_data=(X_test, {'catalog_output': y_test_catalog, 'category_output': y_test_category}),
              epochs=10, batch_size=32)

    # Now you have a trained model that can classify images into catalogs and categories simultaneously.


Processing Images:   0%|          | 0/10286 [00:00<?, ?images/s]

Processing Images:   1%|          | 61/10286 [00:06<10:15, 16.62images/s]  

Error processing D:\PENNY\Gaiat products\Ali_Café_3_In_1_Classic_Instant_Coffee_-_22_Sticks.jpg: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'

Error processing D:\PENNY\Gaiat products\Ali_Café_3_In_1_Instant_French_Coffee_-_20Packs_-24_Sticks.jpg: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'



Processing Images:   3%|▎         | 282/10286 [00:07<01:25, 117.40images/s]

Error processing D:\PENNY\Gaiat products\Bn130_Brown_Paper_Napkin_-_30Cm _40_Packs.jpg: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'



...

Error processing D:\PENNY\Gaiat products\Sharpened_Crème_Caramel_-_72_Pcs_-100g.jpg: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'



Processing Images:  97%|█████████▋| 10010/10286 [01:30<00:02, 124.55images/s]

Error processing D:\PENNY\Gaiat products\Sharpened_Nescafé_Classic_Instant_Coffee_-_12_Bottles.jpg: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'

Error processing D:\PENNY\Gaiat products\Sharpened_Nescafé_Classic_3_In1_Instant_Coffee_-_12Packs_-28_Sticks.jpg: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'



Processing Images: 100%|██████████| 10286/10286 [01:32<00:00, 110.65images/s]


MemoryError: Unable to allocate 11.4 GiB for an array with shape (10197, 224, 224, 3) and data type float64